In [138]:
#Note: Before run, change these two settings manually, and then enter image numbers in this format below:
    # List of subdirectories to check
    #subdirectories = ['35064960', '38551367', '38566504'] #---frontview cameras
    #subdirectories = ['30061123', '33458820', '38146670'] #---backview cameras

    #base_path = f'fvjetson03/RainerVeneer_front/{date}/'#----- frontview
    #base_path = f'fvjetson01/RainerVeneer_back/{date}/'  #----- backview

    #03-14-1752-1757;
    #03-14-1758-1766;
    #03-14-1767-1774
    
#install required libraries
!pip3 install boto3

#import libraries and configure AWS
import boto3
from botocore.exceptions import NoCredentialsError, ClientError

import datetime 
import uuid

import re

#verify AWS credential setup
#necessary actions like s3:ListBucket, s3:GetObject, and s3:PutObject 
#for the relevant resources
def verify_aws_credentials():
    try:
        # Attempt to retrieve AWS credentials
        boto3.setup_default_session()
        print("AWS credentials are configured.")
    except NoCredentialsError:
        print("No AWS credentials were found. Please configure them and try again.")
        
#get caller identity
def print_caller_identity():
    try:
        sts = boto3.client('sts')
        identity = sts.get_caller_identity()
        print(f"Caller Identity: {identity}")
        print ('.\n')
        
        # Extracting specific details from the identity response
        print(f"User ARN: {identity['Arn']}")
        print(f"User ID: {identity['UserId']}")
        
        # The account field represents the AWS account ID
        print(f"Account: {identity['Account']}")
    except NoCredentialsError:
        print("No AWS credentials were found. Please configure them and try again.")
    except Exception as e:
        print(f"An error occurred: {e}")

#check bucket access
import boto3
from botocore.exceptions import ClientError

def check_bucket_access(bucket_name):
    s3 = boto3.client('s3')
    try:
        # Attempt to get the bucket's ACL as a test for access permission
        acl = s3.get_bucket_acl(Bucket=bucket_name)
        #print(f"Successfully retrieved ACL for bucket '{bucket_name}'. " \
        #      f"Permissions appear to be correctly configured.")
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == 'AccessDenied':
            print("Access Denied. You don't have permission to access this bucket.")
        elif e.response['Error']['Code'] == 'NoSuchBucket':
            print("Bucket does not exist.")
        else:
            print(f"Error occurred: {e}")
        return False

    
def list_images(bucket, base_path, date, start_num, end_num):
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    images = []
    
    # Construct the base prefix using the specified date
    # Ensure the base_path ends with a slash
    if not base_path.endswith('/'):
        base_path += '/'
    base_prefix = f'{base_path}{date}/'  # This is where the date was originally added
    
    # List of subdirectories to check
    #subdirectories = ['35064960', '38551367', '38566504'] #---frontview cameras
    subdirectories = ['30061123', '33458820', '38146670'] #---backview cameras
    
    for subdir in subdirectories:
        # Construct the path without repeating the date
        full_prefix = f'{base_path}{subdir}/'  # Removed the second date from the path
        
        for num in range(start_num, end_num + 1):
            for p in ['depth', 'left', 'right']:
                # Construct the key prefix for each image
                key_prefix = f'{full_prefix}{p}_image_{num}.png'
                pages = paginator.paginate(Bucket=bucket, Prefix=key_prefix)
                
                for page in pages:
                    for obj in page.get('Contents', []):
                        images.append(obj['Key'])
                        
    return images

def copy_images(source_bucket, target_bucket, target_folder, images, date, 
                start_num, end_num):
    s3 = boto3.resource('s3')
    
    # Create a subfolder name based on the date and image numbers
    subfolder = f"{date.replace('-', '')}-{start_num}-{end_num}"
    
    # Update target_folder to include the new subfolder
    target_folder = f"{target_folder}/{subfolder}/"

    for img in images:
        # Generate a unique identifier, here using a timestamp or UUID
        unique_identifier = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")  # Or use str(uuid.uuid4())

        # Append the unique identifier to the filename to ensure it's unique
        filename = img.split("/")[-1]
        unique_filename = f"{filename.split('.')[0]}_{unique_identifier}.{filename.split('.')[-1]}"

        target_key = f'{target_folder}{unique_filename}'

        copy_source = {
            'Bucket': source_bucket,
            'Key': img
        }
        s3.meta.client.copy(copy_source, target_bucket, target_key)
        #print(f'Copied {img} to {target_key}')

    
import boto3
from botocore.exceptions import ClientError

def ensure_bucket_exists(bucket_name, region=None):
    s3 = boto3.client('s3', region_name=region)
    try:
        # Try to get the bucket's location to see if it exists
        s3.head_bucket(Bucket=bucket_name)
        #print(f"Bucket '{bucket_name}' already exists.")
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == '404':
            # The bucket does not exist, so try to create it
            print(f"Bucket '{bucket_name}' does not exist. Creating bucket...")
            try:
                if region is None or region == 'us-east-1':
                    # No location constraint necessary if us-east-1
                    s3.create_bucket(Bucket=bucket_name)
                else:
                    # Specify location constraint if region is other than us-east-1
                    s3.create_bucket(Bucket=bucket_name,
                                     CreateBucketConfiguration=
                                     {'LocationConstraint': region})
                print(f"Bucket '{bucket_name}' created successfully.")
            except ClientError as e:
                print(f"Error creating bucket: {e}")
        elif error_code == '403':
            print("Forbidden. Check if you have permission to access or create buckets.")
        else:
            print(f"Error checking bucket: {e}")
            
#list copied files
def list_copied_files(bucket_name, prefix):
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    if 'Contents' in response:
        print(f"Files in {prefix}:")
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print(f"No files found in {prefix}.")


#initialize the S3 client
s3 = boto3.client('s3', region_name='us-west-2') 
# Ensure the region matches your bucket's region

# List buckets
response = s3.list_buckets()
for bucket in response['Buckets']:
    print(bucket['Name'])
    
buckets = s3.list_buckets()
    
from botocore.exceptions import NoCredentialsError

#main function 
def main():

    # User inputs multiple sets, separated by ";"
    user_input_combined = input("Enter the dates, start images, and end images in the format 'MM-DD-start-end'; for multiple inputs separate them by ';': ")
    
    # Remove all spaces from the input
    user_input_combined_no_spaces = re.sub(r'\s+', '', user_input_combined)
    
    # Split the cleaned input by ";"
    user_inputs = user_input_combined_no_spaces.split(';')
    
    # Iterate through each set of inputs
    for user_input in user_inputs:
        # At each input set is processed, print a message
        print(f"---------------------------------------")  
        print(f"---------------------------------------")  
        print(f"Input '{user_input}' executed start.")  
        
        parts = [part.strip() for part in user_input.split('-')]  # Strip each part after splitting 

        #check for input date format 
        if len(parts) != 4:
            print("Invalid input format. Please use 'MM-DD-start-end'.")
            return    

        # Extract and format the date and image numbers
        month, day, start_image_num, end_image_num = parts
        month = month.zfill(2)  # Add leading zero if necessary
        day = day.zfill(2)      # Add leading zero if necessary

        date = f"2024-{month}-{day}"  # Assuming the year is 2024; adjust as necessary
        start_image_num = int(start_image_num)
        end_image_num = int(end_image_num)


        # Check AWS credentials and print caller identity
        #print_caller_identity()    
        #verify_aws_credentials()    

        # Adjust these variables as necessary
        source_bucket = 'forestvision'
        target_bucket = 'forestvision-processedimages'  # Can be the same as source_bucket
        target_folder = 'processed-images-folder'
        #base_path = f'fvjetson03/RainerVeneer_front/{date}/'#----- frontview
        base_path = f'fvjetson01/RainerVeneer_back/{date}/'  #----- backview
        aws_region = 'us-west-2'  # Specify the region for the bucket

        # Ensure the target bucket exists
        ensure_bucket_exists(target_bucket, region=aws_region)

        # Verify AWS Credentials and Access to Bucket
        if not check_bucket_access(source_bucket) or not check_bucket_access(target_bucket):
            print("Exiting due to access issues with one of the buckets.")
            return

        # Copy images
        images = list_images(source_bucket, base_path, date, start_image_num, end_image_num)
        copy_images(source_bucket, target_bucket, target_folder, images, date, start_image_num, end_image_num)
        #list_copied_files(target_bucket, target_folder + '/')

        # After each input set is processed, print a confirmation
        print(f"Input '{user_input}' executed completely.")        
        
if __name__ == '__main__':
    main()

forestvision
forestvision-processedimages
Enter the dates, start images, and end images in the format 'MM-DD-start-end'; for multiple inputs separate them by ';': 03-15-3214-3234
---------------------------------------
---------------------------------------
Input '03-15-3214-3234' executed start.
Input '03-15-3214-3234' executed completely.
